In [1]:
#使用HTML作為打卡介面並用POST提交表單
from flask import Flask, render_template_string, request
from datetime import datetime
import pandas as pd
import os

app = Flask(__name__)

打卡記錄檔案 = '打卡紀錄.csv'

# 初始化打卡記錄檔（如果不存在）
if not os.path.exists(打卡記錄檔案):
    df_init = pd.DataFrame(columns=['姓名', '打卡時間', 'IP'])
    df_init.to_csv(打卡記錄檔案, index=False, encoding='utf-8-sig')

# HTML 模板
template = '''
<!doctype html>
<title>打卡系統</title>
<h2>請輸入姓名打卡</h2>
<form method=post>
  姓名: <input type=text name=name>
  <input type=submit value="打卡">
</form>
{% if message %}
<p>{{ message }}</p>
{% endif %}
'''

@app.route("/", methods=["GET", "POST"])
def CheckIn():
    message = ''
    if request.method == "POST":
        name = request.form["name"]
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        ip = request.headers.get('X-Forwarded-For', request.remote_addr)

        # 讀取原始資料，新增一列
        df = pd.read_csv(打卡記錄檔案, encoding='utf-8-sig')
        new_row = pd.DataFrame([{'姓名': name, '打卡時間': now, 'IP': ip}])
        df = pd.concat([df, new_row], ignore_index=True)

        # 儲存回 CSV
        df.to_csv(打卡記錄檔案, index=False, encoding='utf-8-sig')

        message = f"{name}（{ip}）已於 {now} 打卡成功"
    return render_template_string(template, message=message)

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.16.1.67:5000
Press CTRL+C to quit
172.16.1.6 - - [29/Jun/2025 16:18:16] "GET / HTTP/1.1" 200 -
172.16.1.6 - - [29/Jun/2025 16:18:16] "GET /favicon.ico HTTP/1.1" 404 -
172.16.1.6 - - [29/Jun/2025 16:18:29] "POST / HTTP/1.1" 200 -
172.16.1.36 - - [29/Jun/2025 16:18:42] "GET / HTTP/1.1" 200 -
172.16.1.36 - - [29/Jun/2025 16:18:42] "GET /favicon.ico HTTP/1.1" 404 -
172.16.1.36 - - [29/Jun/2025 16:19:00] "POST / HTTP/1.1" 200 -
172.16.1.36 - - [29/Jun/2025 16:19:04] "GET / HTTP/1.1" 200 -
172.16.1.56 - - [29/Jun/2025 16:19:11] "GET / HTTP/1.1" 200 -
172.16.1.56 - - [29/Jun/2025 16:19:11] "GET /favicon.ico HTTP/1.1" 404 -
172.16.1.47 - - [29/Jun/2025 16:19:15] code 400, message Bad request version ('\x00\x02\x01\x00\x00#\x00\x00\x00\x12\x00\x00\x00')
pc'eçü¼\I°úw¸uÀØþ+å0<|(  êêÀ+À/À,À0Ì©Ì¨ÀÀ   / 5 sÊÊ  ÿ      #      " 400 -
172.16.1.47 - - [29/Jun/2025 16:19:15] code 400

In [ ]:
#使用HTTP的GET方法進行打卡，將姓名作為網址參數提交
from flask import Flask, request
from datetime import datetime
import pandas as pd
import os

app = Flask(__name__)
打卡記錄檔案 = '打卡紀錄.csv'

# 初始化 CSV 檔案（如果不存在）
if not os.path.exists(打卡記錄檔案):
    df_init = pd.DataFrame(columns=['姓名', '打卡時間', 'IP'])
    df_init.to_csv(打卡記錄檔案, index=False, encoding='utf-8-sig')

@app.route("/CheckIn", methods=["GET"])
def CheckIn():
    name = request.args.get('name')  # 取得 URL 參數 ?name=小明
    if not name:
        return "請在網址後加上 ?name=你的名字，例如 /CheckIn?name=小明", 400

    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ip = request.headers.get('X-Forwarded-For', request.remote_addr)  # 支援 proxy 下的真實 IP

    # 讀取現有資料，新增新列
    df = pd.read_csv(打卡記錄檔案, encoding='utf-8-sig')
    new_row = pd.DataFrame([{'姓名': name, '打卡時間': now, 'IP': ip}])
    df = pd.concat([df, new_row], ignore_index=True)

    # 儲存更新回 CSV
    df.to_csv(打卡記錄檔案, index=False, encoding='utf-8-sig')

    return f" {name}（{ip}）已於 {now} 打卡成功！"

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

In [ ]:
#使用HTTP的POST方法進行打卡，將姓名作為Json格式參數提交
from flask import Flask, request
from datetime import datetime
import pandas as pd
import os

app = Flask(__name__)
打卡記錄檔案 = '打卡紀錄.csv'

# 初始化 CSV 檔案（如果不存在）
if not os.path.exists(打卡記錄檔案):
    df_init = pd.DataFrame(columns=['姓名', '打卡時間', 'IP'])
    df_init.to_csv(打卡記錄檔案, index=False, encoding='utf-8-sig')

@app.route('/CheckIn', methods=['POST'])
def CheckIn():
    data = request.get_json()
    if not data or 'name' not in data:
        return {"error": " 請傳入 JSON 格式並包含 name 欄位"}, 400

    name = data['name']
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ip = request.headers.get('X-Forwarded-For', request.remote_addr)

    # 讀取現有資料並加入新資料
    df = pd.read_csv(打卡記錄檔案, encoding='utf-8-sig')
    new_row = pd.DataFrame([{'姓名': name, '打卡時間': now, 'IP': ip}])
    df = pd.concat([df, new_row], ignore_index=True)

    # 儲存回 CSV
    df.to_csv(打卡記錄檔案, index=False, encoding='utf-8-sig')

    return {"message": f" {name}（{ip}）已於 {now} 打卡成功"}

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [ ]:
#模擬打卡1
import requests

url = 'http://127.0.0.1:5000/CheckIn'
response = requests.post(url, json={'name': 'ALAN'})
print(response.json())

In [ ]:
#模擬打卡2
"""
使用Anaconda Powershell Prompt輸入以下指令
Invoke-WebRequest -Uri http://127.0.0.1:5000/CheckIn `
    -Method POST `
    -Body '{"name": "ALAN"}' `
    -ContentType "application/json"
"""